In [1]:
from __future__ import division
import sys
sys.path.insert(0, "~/.local/lib/python3.6/site-packages")

import sympy
from sympy import *
from sympy.physics.quantum import *

def express(a, b, name):
    sym = symbols(name)
    sol = solve(a-sym, b)
    assert len(sol) == 1
    return (sym, sol[0])


def frange(x, y, jump):
    i = 0
    curr = x + i*jump
    
    while curr < y:
        yield curr
        i += 1
        curr = x + i*jump
    

import gmpy2
import mpmath
gmpy2.get_context().precision = 10
mpmath.mp.dps = 10

import numpy
import scipy.linalg

In [2]:
# https://stackoverflow.com/q/59523322/1137334

from sympy.core.operations import AssocOp

def apply_ccr(expr, ccr, reverse=False):
    if not isinstance(expr, Basic):
        raise TypeError("The expression to simplify is not a sympy expression.")
        
    if not isinstance(ccr, Eq):
        if isinstance(ccr, Basic):
            ccr = Eq(ccr, 0)
        else:
            raise TypeError("The canonical commutation relation is not a sympy expression.")
    
    comm = None
    
    for node in preorder_traversal(ccr):
        if isinstance(node, Commutator):
            comm = node
            break
            
    if comm is None:
        raise ValueError("The cannonical commutation relation doesn not include a commutator.")
        
    solutions = solve(ccr, comm)
    
    if len(solutions) != 1:
        raise ValueError("There are more solutions to the cannonical commutation relation.")
        
    value = solutions[0]
    
    A = comm.args[0]
    B = comm.args[1]
    
    if reverse:
        (A, B) = (B, A)
        value = -value
    
    def is_expandable_pow_of(base, expr):
        return isinstance(expr, Pow) \
            and base == expr.args[0] \
            and isinstance(expr.args[1], Number) \
            and expr.args[1] >= 1
    
    
    def walk_tree(expr):
        if isinstance(expr, Number):
            return expr
        
        if not isinstance(expr, AssocOp) and not isinstance(expr, Function):
            return expr.copy()
        
        elif not isinstance(expr, Mul):
            return expr.func(*(walk_tree(node) for node in expr.args))
        
        else:
            args = [arg for arg in expr.args]
            
            for i in range(len(args)-1):
                x = args[i]
                y = args[i+1]
                
                if B == x and A == y:
                    args = args[0:i] + [A*B - value] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if B == x and is_expandable_pow_of(A, y):
                    ypow = Pow(A, y.args[1] - 1)
                    args = args[0:i] + [A*B - value, ypow] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if is_expandable_pow_of(B, x) and A == y:
                    xpow = Pow(B, x.args[1] - 1)
                    args = args[0:i] + [xpow, A*B - value] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if is_expandable_pow_of(B, x) and is_expandable_pow_of(A, y):
                    xpow = Pow(B, x.args[1] - 1)
                    ypow = Pow(A, y.args[1] - 1)
                    args = args[0:i] + [xpow, A*B - value, ypow] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
            
            return expr.copy()
            
    
    return walk_tree(expr)
   

Basic.apply_ccr = lambda self, ccr, reverse=False: apply_ccr(self, ccr, reverse)


In [3]:
# https://stackoverflow.com/q/59524925/1137334

from sympy.core.operations import AssocOp

def apply_operator(expr, eqns):
    if not isinstance(expr, Basic):
        raise TypeError("The expression to simplify is not a sympy expression.")
    
    if not isinstance(eqns, list) and not isinstance(eqns, tuple):
        eqns = (eqns,)
    
    
    rules = []
    
    
    class Rule(object):
        operator = None
        ketSymbol = None
        result = None
        generic = False
    
    
    def is_operator(op):
        return isinstance(op, Operator) \
        or isinstance(op, Dagger) \
        and isinstance(op.args[0], Operator)
    
    
    for eqn in eqns:
        if not isinstance(eqn, Eq):
            raise TypeError("One of the equations is not a valid sympy equation.")
        
        lhs = eqn.lhs
        rhs = eqn.rhs
        
        if not isinstance(lhs, Mul) \
        or len(lhs.args) != 2 \
        or not is_operator(lhs.args[0]) \
        or not isinstance(lhs.args[1], KetBase):
            raise ValueError("The left-hand side has to be an operator applied to a ket.")
        
        rule = Rule()
        rule.operator = lhs.args[0]
        rule.ketSymbol = lhs.args[1].args[0]
        rule.result = rhs
        
        if not isinstance(rule.ketSymbol, Symbol):
            raise ValueError("The left-hand ket has to contain a simple symbol.")
        
        for ket in preorder_traversal(rhs):
            if isinstance(ket, KetBase):
                for symb in preorder_traversal(ket):
                    if symb == rule.ketSymbol:
                        rule.generic = True
                        break
                        
        rules.append(rule)
    
    
    def is_expandable_pow_of(base, expr):
        return isinstance(expr, Pow) \
            and base == expr.args[0] \
            and isinstance(expr.args[1], Number) \
            and expr.args[1] >= 1
            
            
    def is_ket_of_rule(ket, rule):
        if not isinstance(ket, KetBase):
            return False
        
        if rule.generic:
            for sym in preorder_traversal(ket):
                if sym == rule.ketSymbol:
                    return True
            return False
                
        else:
            return ket.args[0] == rule.ketSymbol
    
    
    def walk_tree(expr):
        if isinstance(expr, Number):
            return expr
        
        if not isinstance(expr, AssocOp) and not isinstance(expr, Function):
            return expr.copy()
        
        elif not isinstance(expr, Mul):
            return expr.func(*(walk_tree(node) for node in expr.args))
        
        else:
            args = [arg for arg in expr.args]
            
            for rule in rules:
                A = rule.operator
                ketSym = rule.ketSymbol
                
                for i in range(len(args)-1):
                    x = args[i]
                    y = args[i+1]

                    if A == x and is_ket_of_rule(y, rule):
                        ev = rule.result
                        
                        if rule.generic:
                            ev = ev.subs(rule.ketSymbol, y.args[0])
                        
                        args = args[0:i] + [ev] + args[i+2:]
                        return walk_tree( Mul(*args).expand() )

                    if is_expandable_pow_of(A, x) and is_ket_of_rule(y, rule):
                        xpow = Pow(A, x.args[1] - 1)
                        ev = rule.result
                        
                        if rule.generic:
                            ev = ev.subs(rule.ketSymbol, y.args[0])
                        
                        args = args[0:i] + [xpow, ev] + args[i+2:]
                        return walk_tree( Mul(*args).expand() )
                
            
            return expr.copy()
            
    
    return walk_tree(expr)
   

Basic.apply_operator = lambda self, *eqns: apply_operator(self, eqns)


In [60]:
class OrthogonalKet(State, KetBase):
    
    @classmethod
    def dual_class(self):
        return OrthogonalBra
    
    def _eval_innerproduct(self, bra, **hints):

        if len(self.args) != len(bra.args):
            raise ValueError('Cannot multiply a ket that has a different number of labels.')
            
        for i in range(len(self.args)):
            diff = self.args[i] - bra.args[i]
            diff.simplify()
            
            if diff.is_nonzero:
                return Number(0)
            
            if not diff.is_zero:
                return None
            
        return Number(1)

    
class OrthogonalBra(State, BraBase):
    
    @classmethod
    def dual_class(self):
        return OrthogonalKet



from sympy.printing.pycode import AbstractPythonCodePrinter

def _print_inner_prod(self, expr):
    bra,ket = expr.args
    
    
    if not isinstance(bra, OrthogonalBra) or not isinstance(ket, OrthogonalKet):
        print(bra.func, ket.func)
        raise NotImplementedError('Only implemented for orthogonal states')
    
    if len(ket.args) != len(bra.args):
        raise ValueError('Cannot multiply a ket that has a different number of labels.')

    conditions = []

    for i in range(len(ket.args)):
        conditions.append('{a} == {b}'.format(
            a = self._print(ket.args[i]),
            b = self._print(bra.args[i])
        ))

    return '(1 if {c} else 0)'.format(
        c = ' and '.join(conditions)
    )

AbstractPythonCodePrinter._print_InnerProduct = _print_inner_prod



OKet, OBra = OrthogonalKet, OrthogonalBra

def OBraKet(a,b):
    return InnerProduct(OBra(a), OKet(b))

In [7]:
x, p = Operator('xhat'), Operator('phat')
H = (p**2 + x**2)**2

In [41]:
n = symbols('n', negative=False, integer=True)
t = symbols('t', real=True)

En = n**2 + n + Number(1)/2
En1 = En.subs(n,n+1)

psit = exp(-I*En*t) * Ket(n) + exp(-I*En1*t) * Ket(n+1)

spsit = exp(-I*En*t) * (Ket(n) + exp(-2*I*(n+1)*t) * Ket(n+1))
assert 0 == simplify(psit - spsit)
psit = spsit
psit

exp(-I*t*(n**2 + n + 1/2))*(|n> + exp(-2*I*t*(n + 1))*|n + 1>)

In [51]:
print(latex(psit.subs(Ket(n), Operator('\\ket{n}')).subs(Ket(n+1), Operator('\\ket{n+1}'))))

e^{- i t \left(n^{2} + n + \frac{1}{2}\right)} \left(\ket{n} + e^{- 2 i t \left(n + 1\right)} \ket{n+1}\right)


In [52]:
meanx = Dagger(psit) * x * psit
meanx

(exp(2*I*t*(n + 1))*<n + 1| + <n|)*xhat*(|n> + exp(-2*I*t*(n + 1))*|n + 1>)

In [53]:
meanx = simplify(qapply(meanx.expand()))
meanx

exp(2*I*t*(n + 1))*<n + 1|*xhat*|n> + <n|*xhat*|n> + <n + 1|*xhat*|n + 1> + exp(-2*I*t*(n + 1))*<n|*xhat*|n + 1>

In [54]:
print(latex(meanx.subs(Ket(n), Operator('\\ket{n}')).subs(Ket(n+1), Operator('\\ket{n+1}'))))

e^{2 i t \left(n + 1\right)} {\left\langle n + 1\right|} \hat{x} \ket{n} + {\left\langle n\right|} \hat{x} \ket{n} + {\left\langle n + 1\right|} \hat{x} \ket{n+1} + e^{- 2 i t \left(n + 1\right)} {\left\langle n\right|} \hat{x} \ket{n+1}


In [76]:
a = Operator('ahat')
ad = Dagger(a)

ccr = Eq( Commutator(a, ad),  1 )
down = Eq( a *OKet(n), sqrt(n  )*OKet(n-1) )
up   = Eq( ad*OKet(n), sqrt(n+1)*OKet(n+1) )

x_ = ad + a
p_ = I*(ad - a)

meanx_ = meanx.subs(x,x_)
meanx_

exp(2*I*t*(n + 1))*<n + 1|*(Dagger(ahat) + ahat)*|n> + <n|*(Dagger(ahat) + ahat)*|n> + <n + 1|*(Dagger(ahat) + ahat)*|n + 1> + exp(-2*I*t*(n + 1))*<n|*(Dagger(ahat) + ahat)*|n + 1>

In [77]:
print(latex(meanx_.subs(Ket(n), Operator('\\ket{n}')).subs(Ket(n+1), Operator('\\ket{n+1}'))))

e^{2 i t \left(n + 1\right)} {\left\langle n + 1\right|} \left(\hat{a}^{\dagger} + \hat{a}\right) \ket{n} + {\left\langle n\right|} \left(\hat{a}^{\dagger} + \hat{a}\right) \ket{n} + {\left\langle n + 1\right|} \left(\hat{a}^{\dagger} + \hat{a}\right) \ket{n+1} + e^{- 2 i t \left(n + 1\right)} {\left\langle n\right|} \left(\hat{a}^{\dagger} + \hat{a}\right) \ket{n+1}


In [78]:
meanx_ = expand(meanx_)
meanx_ = apply_operator(meanx_, (up, down))
meanx_ = qapply(meanx_)
meanx_

sqrt(n + 1)*exp(2*I*t)*exp(2*I*n*t) + sqrt(n + 1)*exp(-2*I*t)*exp(-2*I*n*t)

In [79]:
print(latex(meanx_))

\sqrt{n + 1} e^{2 i t} e^{2 i n t} + \sqrt{n + 1} e^{- 2 i t} e^{- 2 i n t}


In [80]:
meanx_ = simplify(meanx_)
meanx_

2*sqrt(n + 1)*cos(2*t*(n + 1))

In [81]:
print(latex(meanx_))

2 \sqrt{n + 1} \cos{\left(2 t \left(n + 1\right) \right)}
